In [1]:
import numpy as np
import pandas as pd
import networkx as nx

In [40]:

from pyvis.network import Network
import networkx as nx


In [2]:
nodes_ale = pd.read_csv(f'edges_Drug_and_pathways_aledronato2.csv default_node.csv')  
edges_ale = pd.read_csv(f'edges_Drug_and_pathways_aledronato2.csv default_edge.csv')


G = nx.Graph()
for _, row in nodes_ale.iterrows():
    G.add_node(row['name'], **row.to_dict())

for _, row in edges_ale.iterrows():
    parts = row['name'].split(' (interacts with) ')
    if len(parts) == 2:
        G.add_edge(parts[0], parts[1], **row.to_dict())

In [4]:
nodes_z = pd.read_csv(f'nodes_gse56815.csv default_node.csv')  
edges_z = pd.read_csv(f'nodes_gse56815.csv default_edge.csv')


G = nx.Graph()
for _, row in nodes_ale.iterrows():
    G.add_node(row['name'], **row.to_dict())

for _, row in edges_ale.iterrows():
    parts = row['name'].split(' (interacts with) ')
    if len(parts) == 2:
        G.add_edge(parts[0], parts[1], **row.to_dict())

In [5]:
nodes_ralo = pd.read_csv(f'nodes_Drug_and_pathways_raloxifeno.csv')  
edges_ralo = pd.read_csv(f'edges_Drug_and_pathways_raloxifeno.csv')


G = nx.Graph()
for _, row in nodes_ale.iterrows():
    G.add_node(row['name'], **row.to_dict())

for _, row in edges_ale.iterrows():
    parts = row['name'].split(' (interacts with) ')
    if len(parts) == 2:
        G.add_edge(parts[0], parts[1], **row.to_dict())

In [14]:
genes_ale = nodes_ale[nodes_ale['type'] == 'Gene'].name.unique().tolist()

In [15]:
genes_z = nodes_z[nodes_z['NodeType'] == 'gene'].name.unique().tolist()

In [16]:
genes_ralo = nodes_ralo[nodes_ralo['type'] == 'Gene'].id.unique().tolist()

In [28]:
genes = genes_ale + genes_z + genes_ralo 

In [29]:
genes = list(set(genes))

In [31]:
import csv 

In [32]:
with open('genes.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(genes)

In [64]:
genes_graph = pd.read_csv('string_interactions_short.tsv', sep='\t')
genes_graph.columns

Index(['#node1', 'node2', 'node1_string_id', 'node2_string_id', 'homology',
       'experimentally_determined_interaction', 'database_annotated',
       'automated_textmining', 'combined_score'],
      dtype='object')

In [102]:
edges_z['source'] = edges_z['name'].str.split(' \(').str[0] 
edges_z['target'] = edges_z['name'].str.split('\) ').str[-1] 

In [108]:
genes_z = edges_z[edges_z['target'] == 'ac zoledronico'].source.unique().tolist()

In [94]:
edges_ale['source'] = edges_ale['name'].str.split(' \(').str[0] 
edges_ale['target'] = edges_ale['name'].str.split('\) ').str[-1] 

In [110]:
genes_ale = edges_ale[edges_ale['target'] == 'ALEDRONATO'].source.unique().tolist()

In [111]:
genes_ralo = edges_ralo[edges_ralo['target'] == 'RALOXIFENO'].source.unique().tolist()

In [144]:
lista_1 = nodes_ale[nodes_ale.log2FC_is_negative == 0].name.unique().tolist()
lista_2 = nodes_z[nodes_z.log2FC_is_negative == 0].name.unique().tolist()
genes_positivos = list(set(lista_1 + lista_2))

In [145]:
lista_1 = nodes_ale[nodes_ale.log2FC_is_negative == 1].name.unique().tolist()
lista_2 = nodes_z[nodes_z.log2FC_is_negative == 1].name.unique().tolist()
genes_negativos = list(set(lista_1 + lista_2))

In [148]:
import networkx as nx
G = nx.Graph()

for _, row in genes_graph.iterrows():
    G.add_edge(row['#node1'], row['node2'])

In [149]:


for n in G.nodes():
    grupos = []
    if n in genes_z:
        grupos.append('Acido Z')
    if n in genes_ale:
        grupos.append('Aledronato')
    if n in genes_ralo:
        grupos.append('Raloxefeno')
    G.nodes[n]['grupo'] = ','.join(grupos) if grupos else 'Nenhum'
    
    if n in genes_negativos:
        G.nodes[n]['expresso'] = 'Negativamente'
    elif n in genes_positivos:
        G.nodes[n]['expresso'] = 'Positivamente'
    else:
        G.nodes[n]['expresso'] = 'Não expresso'


In [150]:
componentes = list(nx.connected_components(G))
maior_componente = max(componentes, key=len)
G = G.subgraph(maior_componente).copy()


In [306]:
from pyvis.network import Network

def plotar_grafo_pyvis_custom(G, filename='grafo_padrao.html'):
    net = Network(notebook=True, width='1500px', height='1000px', bgcolor='#d3d3d3', font_color='black')
    
    # Definir cores por grupo
    cores = {
        'Acido Z': 'red',
        'Aledronato': 'green',
        'Raloxefeno': 'blue',
        'Acido Z,Aledronato': 'purple',
        'Acido Z,Raloxefeno': 'orange',
        'Aledronato,Raloxefeno': 'cyan',
        'Acido Z,Aledronato,Raloxefeno': 'black',
        'Nenhum': '#fff9b1'
    }
    
    # Adiciona nós com cor baseada no atributo
    for n, d in G.nodes(data=True):
        grupo = d.get('grupo', 'Nenhum')
        cor = cores.get(grupo, '#fff9b1')
        net.add_node(n, label=n, color=cor, shape='ellipse', title=f"Grupo: {grupo}")
    
    # Adiciona arestas
    for u, v in G.edges():
        net.add_edge(u, v)
    
    net.show_buttons(filter_=['physics'])
    net.show(filename)



In [307]:
plotar_grafo_pyvis_custom(G)

grafo_padrao.html


In [308]:

import json

with open('positions.json', 'r') as f:
    pos = json.load(f)


In [316]:
def plotar_grafo_pyvis_custom(G, pos, filename='grafo_padrao.html'):
    net = Network(notebook=True, width='1500px', height='1000px', bgcolor='#d3d3d3', font_color='black')
    
    # Definir cores por grupo
    cores = {
        'Acido Z': 'red',
        'Aledronato': 'green',
        'Raloxefeno': 'blue',
        'Acido Z,Aledronato': 'purple',
        'Acido Z,Raloxefeno': 'orange',
        'Aledronato,Raloxefeno': 'cyan',
        'Acido Z,Aledronato,Raloxefeno': 'black',
        'Nenhum': '#fff9b1'
    }
    
    # Adiciona nós com cor baseada no atributo e posição fixa
    for n, d in G.nodes(data=True):
        grupo = d.get('grupo', 'Nenhum')
        cor = cores.get(grupo, '#fff9b1')
        
        x = pos[str(n)]['x']
        y = pos[str(n)]['y']
        net.add_node(n, label=n, color=cor, shape='ellipse', title=f"Grupo: {grupo}", x=x, y=y)
    
    # Adiciona arestas
    for u, v in G.edges():
        net.add_edge(u, v)
    
    # Desativa a física para manter as posições fixas
    net.toggle_physics(False)
    #net.show_buttons(filter_=['physics'])
    net.show(filename)

In [317]:
plotar_grafo_pyvis_custom(G, pos)

grafo_padrao.html


In [320]:
from pyvis.network import Network
import networkx as nx

def plotar_grafo_pyvis_custom(G, pos, filename='grafo_padrao_unicor_com_genes_expressos.html'):
    net = Network(notebook=True, width='1500px', height='1000px', bgcolor='#d3d3d3', font_color='black')

    # Listas para separar nós normais e especiais
    nos_normais = []
    nos_especiais = []

    for n, d in G.nodes(data=True):
        grupo = d.get('grupo', 'Nenhum')
        expresso = d.get('expresso', 'Não expresso')

        # Cor
        if grupo != 'Nenhum':
            cor = 'green'  # alvo do medicamento
        elif expresso == 'Positivamente':
            cor = 'red'
        elif expresso == 'Negativamente':
            cor = 'red'
        elif expresso != 'Não expresso':
            cor = 'red'    # expresso sem grupo → vermelho
        else:
            cor = '#fff9b1'  # padrão

        # Shape
        if expresso == 'Positivamente':
            shape = 'triangle'
        elif expresso == 'Negativamente':
            shape = 'triangleDown'
        elif expresso != 'Não expresso':
            shape = 'diamond'
        elif grupo != 'Nenhum':
            shape = 'ellipse'
        else:
            shape = 'box'

        # Tooltip
        title = f"Grupo: {grupo}<br>Expresso: {expresso}"

        # Posição fixa
        x = pos[str(n)]['x']
        y = pos[str(n)]['y']

        # Critério para nó especial: formato ou cor diferente do padrão
        if shape != 'box' or cor != '#fff9b1':
            nos_especiais.append((n, cor, shape, title, x, y))
        else:
            nos_normais.append((n, cor, shape, title, x, y))

    # Adiciona primeiro os nós normais
    for n, cor, shape, title, x, y in nos_normais:
        net.add_node(n, label=n, color=cor, shape=shape, title=title, x=x, y=y)

    # Depois adiciona os nós especiais (ficam “por cima”)
    for n, cor, shape, title, x, y in nos_especiais:
        net.add_node(n, label=n, color=cor, shape=shape, title=title, x=x, y=y)

    # Adiciona arestas
    for u, v in G.edges():
        net.add_edge(u, v)

    net.toggle_physics(False)  # Desativa física para manter posições
    net.show_buttons(filter_=['physics'])
    net.show(filename)

In [321]:
plotar_grafo_pyvis_custom(G, pos)

grafo_padrao_unicor_com_genes_expressos.html


In [326]:
def plotar_grafo_pyvis_custom(G, pos, filename='centralidade.html'):
    net = Network(notebook=True, width='1500px', height='1000px', bgcolor='#d3d3d3', font_color='black')
    
    # Calcula centralidade de autovetor
    #centralidade = nx.eigenvector_centrality(G)
    centralidade = nx.closeness_centrality(G)
    #centralidade = nx.betweenness_centrality(G)
    
    # Normaliza para definir tamanho (ex.: entre 10 e 50)
    max_centralidade = max(centralidade.values()) if centralidade else 1
    
    for n, d in G.nodes(data=True):
        grupo = d.get('grupo', 'Nenhum')
        expresso = d.get('expresso', 'Não expresso')
        
        # Formato
        if expresso == 'Positivamente':
            shape = 'triangle'
        elif expresso == 'Negativamente':
            shape = 'triangleDown'
        elif expresso != 'Não expresso':
            shape = 'diamond'
        elif grupo != 'Nenhum':
            shape = 'ellipse'
        else:
            shape = 'box'
        
        # Cor
        if expresso != 'Não expresso':
            cor = 'red'
        elif grupo != 'Nenhum':
            cor = 'green'
        else:
            cor = '#fff9b1'
        
        # Tamanho baseado na centralidade
        size = 10 + (centralidade[n] / max_centralidade) * 50  # escala 10–50
        
        # Tooltip
        title = f"Grupo: {grupo}<br>Expresso: {expresso}<br>Centralidade: {centralidade[n]:.4f}"
        
        # Posição fixa
        x = pos[str(n)]['x']
        y = pos[str(n)]['y']
        net.add_node(n, label=n, color=cor, shape=shape, size=size, title=title, x=x, y=y)
    
    # Adiciona arestas
    for u, v in G.edges():
        net.add_edge(u, v)
    
    net.toggle_physics(False)  # Desativa física para manter posições
    net.show_buttons(filter_=['physics'])
    net.show(filename)

In [327]:
plotar_grafo_pyvis_custom(G, pos)

centralidade.html


In [328]:
def plotar_grafo_pyvis_custom(G, pos, filename='grafo_genes_expresso_fora_alcance.html'):
    net = Network(notebook=True, width='1500px', height='1000px', bgcolor='#d3d3d3', font_color='black')
    
    # 1. Lista de genes em grupo
    genes_em_grupo = [n for n, d in G.nodes(data=True) if d.get('grupo', 'Nenhum') != 'Nenhum']
    
    for n, d in G.nodes(data=True):
        grupo = d.get('grupo', 'Nenhum')
        expresso = d.get('expresso', 'Não expresso')
        
        # Shape conforme expressão/grupo
        if expresso == 'Positivamente':
            shape = 'triangle'
        elif expresso == 'Negativamente':
            shape = 'triangleDown'
        elif expresso != 'Não expresso':
            shape = 'diamond'
        elif grupo != 'Nenhum':
            shape = 'ellipse'
        else:
            shape = 'box'
        
        # 2. Se gene está expresso e NÃO está próximo de gene em grupo
        cor = '#fff9b1'  # padrão
        if grupo != 'Nenhum':
            cor = 'blue'
        elif expresso != 'Não expresso':
            # Verifica vizinhança até 2 passos
            esta_proximo = False
            for gene_grupo in genes_em_grupo:
                try:
                    if nx.has_path(G, n, gene_grupo) and nx.shortest_path_length(G, n, gene_grupo) <= 2:
                        esta_proximo = True
                        break
                except nx.NetworkXNoPath:
                    continue
            if not esta_proximo:
                cor = 'red'
            else:
                cor = 'green'  # Se quiser outra cor para os próximos, pode mudar aqui
        
        # Tooltip
        title = f"Grupo: {grupo}<br>Expresso: {expresso}"
        
        # Posição fixa
        x = pos[str(n)]['x']
        y = pos[str(n)]['y']
        net.add_node(n, label=n, color=cor, shape=shape, title=title, x=x, y=y)
    
    # Adiciona arestas
    for u, v in G.edges():
        net.add_edge(u, v)
    
    net.toggle_physics(False)  # Desativa física para manter posições
    net.show_buttons(filter_=['physics'])
    net.show(filename)

In [329]:
plotar_grafo_pyvis_custom(G, pos)

grafo_genes_expresso_fora_alcance.html
